In [1]:
from analysisdatalink.datalink_ext import AnalysisDataLinkExt as AnalysisDataLink
from meshparty import trimesh_io, trimesh_vtk, skeletonize, mesh_filters
from dotenv import load_dotenv
import numpy as np
import os


searching for scad in: /Users/caseyschneider-mizell/anaconda3/envs/jupyter_space/bin:/Users/caseyschneider-mizell/.gem/ruby/2.7.0/bin:/usr/local/opt/ruby/bin:/Users/drpoo/anaconda3/bin:/anaconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Applications/VMware Fusion.app/Contents/Public:/Library/TeX/texbin:/opt/X11/bin:/Applications/OpenSCAD.app/Contents/MacOS


Skeletonization needs to handle processes differently from cell bodies. The current implementation of skeletonization uses the soma center as a way to approximate the process.

In [5]:
data_version = 185
soma_table = 'soma_valence_v2'
dataset_name = 'pinky100'
sql_db_base = "USE_THE_VALUE_GIVEN_BEFORE"

dl = AnalysisDataLink(dataset_name=dataset_name,
                      sqlalchemy_database_uri=sql_db_base,
                      materialization_version=data_version,
                      verbose=False)

soma_df = dl.query_cell_types(soma_table)

We set up a meshmeta to assist with loading meshes. For those cells with soma in the volume, I highly recommend using the pre-generated ones and putting them in the folder you specify for `disk_cache_path`. If you need to download other meshes for the v185 data, I've set it up here to download them as well. Meshparty will default to meshes on disk for the same root id.

In [52]:
mm = trimesh_io.MeshMeta(cv_path="precomputed://gs://microns_public_datasets/pinky100_v185/seg", 
                         cache_size=0, disk_cache_path='meshes/', map_gs_to_https=True)

We load the mesh based on its root id, although we could also specify a filename.

In [9]:
oid = 648518346349539789
mesh_raw = mm.mesh(seg_id=oid)

Meshes have a lot of dangling nodes and faces due to internal holes in the segmentation. These will mess with skeletonization and also sometimes "capture" annotations like synapses, so we want to filter those out. The best way to do that is to keep the largest connected component on the mesh. This is one of several filter functions in mesh_filters that returns a boolean for every node in a mesh. We can then pass that to "apply_mask" to make a new mesh with the old one masked out.

In [ ]:
in_comp = mesh_filters.filter_largest_component(mesh_raw)
mesh = mesh_raw.apply_mask(in_comp)

We now want to skeletonize the mesh. The default parameters mostly work for this, but if we want the soma to be handled properly we need to pass it (in nm, not voxels!) as a `soma_pt`.

In [11]:
voxel_resolution = np.array([4,4,40])
soma_pt = soma_df.query(f'pt_root_id == {oid}')['pt_position'].values[0] * voxel_resolution

sk = skeletonize.skeletonize_mesh(mesh, soma_pt, verbose=False)

Just to visualize that everything has worked, let's look at the following:

In [30]:
ma = trimesh_vtk.mesh_actor(mesh, opacity=0.6)
ska = trimesh_vtk.skeleton_actor(sk, line_width=4, color=[0,0,0])

trimesh_vtk.render_actors([ma, ska])

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x131c8f468

Skeletons derived from meshes have a lot of handy functions for navigating them. For example, `sk.branch_points` is a property with the node indices of every branch point, and `sk.downstream_nodes` will return the indices of every node downstream of its argument.

In [20]:
sk_node = sk.branch_points[6]
sk_nodes_downstream = sk.downstream_nodes(sk_node)

Now we want to map skeleton points back to the original meshes.

sk.mesh_to_skel_map is the length of the original (unfiltered) mesh's node list. For each mesh node, it gives the closest skeleton node as found by skeletonization (with -1 for those nodes not in the skeletonization). We want to know which mesh indices are associated with the skeleton indices downstream of that branch point above. For the first step, we use `np.isin` to figure out which mesh nodes map to skeleton nodes in that downstream set. Since those are in the indexing of the unmasked mesh, we then want to filter those down to the cleaner masked mesh using `filter_unmasked_boolean`. The result, `mesh_downstream`, is an array with a boolean element for every node in the filtered mesh where True means that it is downstream.

In [ ]:
mesh_downstream_unfiltered = np.isin( sk.mesh_to_skel_map, sk_nodes_downstream )
mesh_downstream = mesh.filter_unmasked_boolean( mesh_nodes_downstream_unfiltered )

For visualization purposes, we're going to make a mesh that colors upstream faces a different color than the downstream ones. Here, we do this by giving a different interger to each type of mesh node (upstream/downstream) and then make a list of RGB colors for every node based on this category.

In [50]:
clrs = np.zeros(mesh.n_vertices, dtype=int)
clrs[mesh_downstream] = 1 

clr_lookup = np.array([(0.5, 0.3, 0.3), (0.8, 0.2, 0.1)])
clr_vals = clr_lookup[clrs]

cma = trimesh_vtk.mesh_actor(mesh, opacity=0.6, vertex_colors=clr_vals)
pa = trimesh_vtk.point_cloud_actor([soma_pt], color=(0,0,0), size=5000)

trimesh_vtk.render_actors([cma, pa])

Let's look at synapses as well

In [58]:
dl = AnalysisDataLink(dataset_name=dataset_name,
                      sqlalchemy_database_uri=sql_db_base,
                      materialization_version=data_version,
                      verbose=False)


syn_df = dl.query_synapses('pni_synapses_i3', post_ids=[oid])

We get a list of the positions of every synapse and use the kdtree to look up the closest node on the clean mesh. In order to look up the associated data, this example maps things back to the dataframe so we can look up all of the associated synapse information.

In [86]:
syn_pts = np.vstack(syn_df['ctr_pt_position'].values) * voxel_resolution

dist, mind = mesh.kdtree.query(syn_pts)
syn_df['mesh_ind'] = mind
syn_df['is_downstream'] = mesh_downsteam[mind]

print("The fraction of all inputs onto the downstream branch is: ", len( syn_df.query('is_downstream == True') ) / len(syn_df) )

The fraction of all inputs onto the downstream branch is:  0.09714285714285714


Just to visualize that we're doing this correctly, let's plot only the location of those synapses on the downstream branches.

In [89]:
syn_pts_downstream = syn_pts[syn_df['is_downstream']]
syna = trimesh_vtk.point_cloud_actor(syn_pts_downstream, color=[0.333, 0.702, 0.698], size=500)
trimesh_vtk.render_actors([cma, pa, syna])

(vtkRenderingOpenGL2Python.vtkOpenGLRenderer)0x16b5c7d68